In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from scipy import stats
import krippendorff
import matplotlib.pyplot as plt

In [2]:
path = "experiments/Chat Prompt Mode, Think Mode [30:]"
results_df = pd.read_parquet(path+"/results.parquet")

print(len(results_df))
results_df["llm_decision"].value_counts()
print(len(results_df["combination_id"].unique()))
# results_df.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)
results_df

74261
688


,experiment_id,combination_id,datapoint_idx,dataset_index,true_outcome,predicted_outcome,llm_decision,reasoning_length,processing_time,total_tokens,tokens_per_second,username,user_decision,chat_id,tracker_len,final_prompt_type_category,final_prompt_type_value,think_mode_category,think_mode_value,include_datapoint_info
0,"Chat Prompt Mode, Think Mode [30:]",0,20,71076,Cancellation,Cancellation,Cancellation,0,0.771478,593,768.654753,007,confirm_datapoint,0072025-06-03 13:41:23:486908,4,ai_trust_emotional,I'm so excited to hear your thoughts on this! ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
1,"Chat Prompt Mode, Think Mode [30:]",0,21,78335,Check-Out,Check-Out,Check-Out,0,0.265290,1021,3848.625093,007,confirm_datapoint,0072025-06-03 13:42:02:814567,7,ai_trust_emotional,I'm so excited to hear your thoughts on this! ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
2,"Chat Prompt Mode, Think Mode [30:]",0,22,80601,Check-Out,Cancellation,Cancellation,0,0.339787,1250,3678.777745,007,confirm_datapoint,0072025-06-03 13:42:55:646696,9,ai_trust_emotional,I'm so excited to hear your thoughts on this! ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
3,"Chat Prompt Mode, Think Mode [30:]",0,23,81743,Cancellation,Cancellation,Cancellation,0,0.261425,597,2283.637595,007,confirm_datapoint,0072025-06-03 13:43:59:126075,4,ai_trust_emotional,I'm so excited to hear your thoughts on this! ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
4,"Chat Prompt Mode, Think Mode [30:]",0,25,84583,Check-Out,Check-Out,Check-Out,0,0.251189,591,2352.805630,007,confirm_datapoint,0072025-06-03 13:44:33:156199,4,ai_trust_emotional,I'm so excited to hear your thoughts on this! ...,direct,Decide! Only write #CANCELLATION# or #CHECK-OU...,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74256,"Chat Prompt Mode, Think Mode [30:]",687,10,30195,Check-Out,Check-Out,Check-Out,2904,4.460648,1514,339.412601,028,decline_datapoint,0282025-06-13 10:56:15:094004,8,ai_neutral_rational,Let's proceed systematically - considering eve...,cot,Deliberate sequentially over each component of...,Yes
74257,"Chat Prompt Mode, Think Mode [30:]",687,14,37601,Check-Out,Check-Out,Check-Out,2434,3.738819,1057,282.709566,028,decline_datapoint,0282025-06-13 10:59:18:542404,3,ai_neutral_rational,Let's proceed systematically - considering eve...,cot,Deliberate sequentially over each component of...,Yes
74258,"Chat Prompt Mode, Think Mode [30:]",687,15,55780,Cancellation,Check-Out,Check-Out,2675,3.955850,1178,297.786795,028,decline_datapoint,0282025-06-13 11:00:15:375427,3,ai_neutral_rational,Let's proceed systematically - considering eve...,cot,Deliberate sequentially over each component of...,Yes
74259,"Chat Prompt Mode, Think Mode [30:]",687,19,68689,Cancellation,Cancellation,Cancellation,2612,4.119168,1444,350.556225,028,confirm_datapoint,0282025-06-13 11:02:45:915952,6,ai_neutral_rational,Let's proceed systematically - considering eve...,cot,Deliberate sequentially over each component of...,Yes


In [50]:
# path = "experiments/prompt_variations_exclusion_shap-lime"
# results_df2 = pd.read_parquet(path+"/results.parquet")
# results_df2.drop(["experiment_id", "reasoning_length", "tokens_per_second", "total_tokens", "final_prompt_type_value", "processing_time"], axis=1, inplace=True)
#
# print(results_df2["final_prompt_type_category"].unique())
# results_df2["combination_id"] = results_df2["combination_id"].apply(lambda x: x+100)
# results_df2["xai_exclusion_pdp"] = False
# results_df2["xai_exclusion_histogram"] = False
#
#
# results_df["xai_exclusion_shap"] =False
# results_df["xai_exclusion_lime_explanation"] = False
# results_df = pd.concat([results_df, results_df2], ignore_index=True)
# results_df

In [16]:
# results_df["method_order"] = results_df["method_order"].apply(lambda x: str(x)[1:-1])
# results_df

In [4]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
summary


columns = summary.columns
for column in columns:
    if column in ['n_datapoints', 'percent_correct', 'percent_agree', 'corr_correct',
       'corr_agree', 'false_positive_rate', 'false_negative_rate',"think_mode_value"]:
        continue

    marginal = column#"final_prompt_type_category"
    comparison = analyzer.compare_parameters(marginal,True)
    comparison = comparison.round(4).drop(["n_combinations"], axis=1)
    comparison


    df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
    df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                             "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                             "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                             "avg_false_positive_rate": "$\\overline{FP}$",
                             "final_prompt_type_category":"Prompt category"}, inplace=True)
    table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                            column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
    table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
    print(table_str)
    print()

#\hspace*{-1.5cm}


# df_t = df_clean.T
# df_t = df_t.set_axis(df_t.iloc[0], axis=1).iloc[1:]
# df_t.columns = df_t.columns.str.replace('_', ' ')
#
# print(df_t.to_latex(escape=True, float_format="%.2f", caption=marginal.replace("_", " "),
#                                                              column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c"))



\begin{table}
\caption{final prompt type category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
Prompt category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
ai neutral emotional & 0.47 & 68.66+-4.47 & 79.27+-10.58 & 0.30+-0.12 & 0.58+-0.22 & 0.22+-0.07 & 0.09+-0.07 \\
ai neutral rational & 0.62 & 69.43+-4.21 & 84.71+-8.08 & 0.33+-0.11 & 0.70+-0.16 & 0.18+-0.06 & 0.12+-0.05 \\
\bottomrule
\end{tabular}
\end{table}


\begin{table}
\caption{think mode category}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
think mode category & $\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{

In [53]:
from rixeval.analysis import ResultAnalyzer
pd.set_option("display.precision", 4)

results_df_temp = results_df#[results_df]

analyzer = ResultAnalyzer(results_df_temp)
summary = analyzer.analyze_all_combinations()
comparison = analyzer.compare_parameters(None,True)
comparison = comparison.round(4).drop(["n_combinations"], axis=1)
comparison


df_clean = comparison#comparison.drop(["n_combinations"], axis=1)#
df_clean.rename(columns={"krippendorff_alpha": "$\\alpha$", "avg_percent_correct": "Avg. \% correct",
                         "avg_percent_agree": "Avg. \% agree", "avg_corr_agree": "$r_{\\mathrm{agree}}$",
                         "avg_corr_correct":"$r_{\\mathrm{correct}}$", "avg_false_negative_rate":"$\\overline{FN}$",
                         "avg_false_positive_rate": "$\\overline{FP}$",
                         "final_prompt_type_category":"Prompt category"}, inplace=True)
table_str = df_clean.to_latex(escape=False, caption=marginal.replace("_", " "),float_format="%.2f", index=False,
                        column_format="l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c")
table_str = table_str.replace("_", " ").replace("r ", "r_").replace(r"\begin{tabular}", "\hspace*{-2.5cm}\n\\begin{tabular}")
print(table_str)


\begin{table}
\caption{xai exclusion histograms}
\hspace*{-2.5cm}
\begin{tabular}{l @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c @{\hspace{1em}} c}
\toprule
$\alpha$ & Avg. \% correct & Avg. \% agree & $r_{\mathrm{correct}}$ & $r_{\mathrm{agree}}$ & $\overline{FP}$ & $\overline{FN}$ \\
\midrule
0.51 & 67.67+-7.17 & 84.50+-14.03 & 0.28+-0.15 & 0.68+-0.30 & 0.20+-0.07 & 0.12+-0.09 \\
\bottomrule
\end{tabular}
\end{table}



In [11]:
import pandas as pd

data = {
    'XAI Method': ['SHAP', 'SHAP', 'LIME Explanation', 'LIME Explanation', 'PDP', 'PDP',
                   'Counterfactual1', 'Counterfactual1', 'Histograms', 'Histograms', 'All'],
    'Exclusion': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes'],
    'Krippendorffs Alpha': [0.560, 0.682, 0.656, 0.585, 0.647, 0.588, 0.634, 0.616, 0.615, 0.618, 0.509],
    'Avg % Correct': [67.051, 67.643, 67.812, 66.882, 67.656, 67.038, 66.289, 68.405, 67.350, 67.344, 67.667],
    'Avg % Correct Var': [5.795, 4.979, 4.999, 5.754, 4.646, 6.063, 5.641, 4.948, 5.107, 5.697, 7.169],
    'Avg % Agree': [85.514, 90.482, 89.440, 86.556, 88.854, 87.142, 87.461, 88.535, 87.949, 88.047, 84.500],
    'Avg % Agree Var': [12.484, 10.281, 10.755, 12.413, 10.349, 12.859, 11.377, 11.996, 11.569, 11.835, 14.032],
    'Avg Corr Correct': [0.266, 0.297, 0.295, 0.268, 0.285, 0.279, 0.251, 0.313, 0.282, 0.282, 0.283],
    'Avg Corr Correct Var': [0.146, 0.110, 0.122, 0.137, 0.128, 0.132, 0.142, 0.109, 0.124, 0.136, 0.152],
    'Avg Corr Agree': [0.705, 0.811, 0.790, 0.726, 0.773, 0.743, 0.744, 0.772, 0.757, 0.759, 0.681],
    'Avg Corr Agree Var': [0.260, 0.201, 0.213, 0.258, 0.218, 0.257, 0.239, 0.237, 0.236, 0.241, 0.298]
}

df = pd.DataFrame(data)

# Display the DataFrame
df

,XAI Method,Exclusion,Krippendorffs Alpha,Avg % Correct,Avg % Correct Var,Avg % Agree,Avg % Agree Var,Avg Corr Correct,Avg Corr Correct Var,Avg Corr Agree,Avg Corr Agree Var
0,SHAP,No,0.560,67.051,5.795,85.514,12.484,0.266,0.146,0.705,0.260
1,SHAP,Yes,0.682,67.643,4.979,90.482,10.281,0.297,0.110,0.811,0.201
2,LIME Explanation,No,0.656,67.812,4.999,89.440,10.755,0.295,0.122,0.790,0.213
3,LIME Explanation,Yes,0.585,66.882,5.754,86.556,12.413,0.268,0.137,0.726,0.258
4,PDP,No,0.647,67.656,4.646,88.854,10.349,0.285,0.128,0.773,0.218
5,PDP,Yes,0.588,67.038,6.063,87.142,12.859,0.279,0.132,0.743,0.257
6,Counterfactual1,No,0.634,66.289,5.641,87.461,11.377,0.251,0.142,0.744,0.239
7,Counterfactual1,Yes,0.616,68.405,4.948,88.535,11.996,0.313,0.109,0.772,0.237
8,Histograms,No,0.615,67.350,5.107,87.949,11.569,0.282,0.124,0.757,0.236
9,Histograms,Yes,0.618,67.344,5.697,88.047,11.835,0.282,0.136,0.759,0.241


In [5]:
df_no = df[df['Exclusion'] == 'No']
df_no["Krippendorffs Alpha"].mean()

np.float64(0.6224000000000001)